### Installing and Importing required libraries

In [12]:
#Importing required libraries
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from datetime import datetime
from scipy.stats import ttest_ind
from scipy.optimize import minimize_scalar
from sklearn.model_selection import train_test_split

#Global setting for ignoring warnings
warnings.filterwarnings("ignore")

### Data Ingestion

In [13]:
#Loading the dataset
data_url = "https://raw.githubusercontent.com/ormarketing/b2b/master/data.csv"
pricing_data = pd.read_csv(data_url)

### Data Exploration

In [14]:
#Observing the column names
print("Column Names : ", pricing_data.columns)

#Observing the dataset characteristics
print("\nDataset Summmary :\n ", pricing_data.describe())

#Observing the data
print("\nSample Data : \n", pricing_data.head(10))

#Checking the data types of columns
print("\nData types of columns:")
print(pricing_data.dtypes)

#Checking for missing values
print("\nMissing values:")
print(pricing_data.isnull().sum())

#Exploinge unique values in categorical columns
print("\nUnique values in categorical columns:")
categorical_columns = ['Type', 'Stage', 'Billing Plan']
for col in categorical_columns:
    print(f"{col}: {pricing_data[col].unique()}")

#Calculate the frequency count of type
print("\nProduct/Channel frequency count:")
print(pricing_data['Type'].value_counts())

Column Names :  Index(['ID', 'Close Date', 'Created Date', 'New Logo', 'Type', 'Stage',
       'Billing Plan', ' ACV ', ' Discount Amount ', ' Amount ',
       ' Net Amount '],
      dtype='object')

Dataset Summmary :
                  ID     New Logo
count  1505.000000  1505.000000
mean    753.000000     0.465781
std     434.600391     0.498993
min       1.000000     0.000000
25%     377.000000     0.000000
50%     753.000000     0.000000
75%    1129.000000     1.000000
max    1505.000000     1.000000

Sample Data : 
    ID Close Date Created Date  New Logo                         Type  \
0   1    5/26/17       5/4/17         1       Direct - Cloud Product   
1   2   12/29/16     11/29/16         1  Partner - OnPremise Product   
2   3   12/29/16      12/9/16         1      Partner - Cloud Product   
3   4     9/9/16      4/20/16         1       Direct - Cloud Product   
4   5    9/22/16       9/1/16         1  Partner - OnPremise Product   
5   6    12/6/16      9/12/16         1  P

### Data Pre-processing

In [15]:
#Stripping leading and trailing whitespaces from column names
pricing_data.columns = [col.strip() for col in pricing_data.columns]

#Encoding the 'Stage' variable as binary (0: Lost, 1: Won)
pricing_data['Stage'] = np.where(pricing_data['Stage'].str.contains('Won'), 1, 0)

#Creating product channels based on 'Type' variable
pricing_data['Partner Cloud'] = np.where(pricing_data['Type'] == 'Partner - Cloud Product', 1, 0)
pricing_data['Partner Prem'] = np.where(pricing_data['Type'] == 'Partner - OnPremise Product', 1, 0)
pricing_data['Direct Cloud'] = np.where(pricing_data['Type'] == 'Direct - Cloud Product', 1, 0)
pricing_data['Direct Prem'] = np.where(pricing_data['Type'] == 'Direct - OnPremise Product', 1, 0)

#Making sure entries are numeric
pricing_data['Amount'] = pd.to_numeric(pricing_data['Amount'], errors = 'coerce')
pricing_data['Net Amount'] = pd.to_numeric(pricing_data['Net Amount'], errors = 'coerce')
pricing_data['Discount Amount'] = pricing_data['Amount'] - pricing_data['Net Amount']
pricing_data['Depth'] = pricing_data['Discount Amount'] / pricing_data['Amount']

#Preprocessing the Billing Plan variable
billing_plan_mapping = {'3 Years': 3, '4 Years': 4, '2 Years': 2, 'Annual': 1, '5 Years': 5, '3 Months': 0.25, '9 Months': 0.75, 'Quarterly': 0.25}
pricing_data['Billing_Plan'] = pricing_data['Billing Plan'].map(billing_plan_mapping)

#Creating a field to include number of days the plan was active
def convert_to_datetime(date_string):
    return datetime.strptime(date_string, '%m/%d/%y')
pricing_data['Close Date'] = pricing_data['Close Date'].apply(convert_to_datetime)
pricing_data['Created Date'] = pricing_data['Created Date'].apply(convert_to_datetime)

pricing_data['Active Days'] = (pricing_data['Close Date'] - pricing_data['Created Date']).dt.days
pricing_data['Active Days'] = pricing_data['Active Days'].astype(int)

#Creating a constant value column
pricing_data['constant'] = 1

### End-of-Quarter effect

In [16]:
#Testing the hypothesis of End-of-the-Quarter effect
pricing_data['Date'] = pd.to_datetime(pricing_data['Close Date'])
pricing_data['Quarter'] = pricing_data['Date'].dt.quarter
pricing_data['Year'] = pricing_data['Date'].dt.year
pricing_data['End of Quarter'] = np.where((pricing_data['Date'].dt.month % 3 == 0) & (pricing_data['Date'].dt.day >= 20), 1, 0)
#We consider the last ten days of each quarter as the End-of-Quarter

#Creating different datasets for hypothesis testing
end_of_quarter_discounts = pricing_data.loc[pricing_data['End of Quarter'] == 1, 'Discount Amount']
non_end_of_quarter_discounts = pricing_data.loc[pricing_data['End of Quarter'] == 0, 'Discount Amount']

#Dropping NAs
end_of_quarter_discounts = end_of_quarter_discounts.dropna()
non_end_of_quarter_discounts = non_end_of_quarter_discounts.dropna()

#Using the t-test to test for End-of-Quarter effect
t_stat, p_value = ttest_ind(end_of_quarter_discounts, non_end_of_quarter_discounts)
print("\nHypothesis Test Results")
print(f"T-statistic : {t_stat}")
print(f"P-value : {p_value}\n")


Hypothesis Test Results
T-statistic : 6.34549984203434
P-value : 2.9553213746096387e-10



We observe a low p-value, less than 0.05, implying that there does exist an 'End-of-Quarter' effect, meaning that the salespeople do tend to offer steeper discounts near the end of each quarter.

### Estimating the models

In [17]:
#Splitting the data into Estimation and Holdout Samples
estimation_df = pricing_data.sample(frac = 0.8, random_state = 5315)
holdout_df = pricing_data.drop(estimation_df.index)

print("\nLogit Model Summaries : \n")

#Estimating Logit Models for each Product Channel
logit_models = {}
for channel in ['Overall', 'Partner Prem', 'Direct Prem', 'Partner Cloud', 'Direct Cloud']:
    if channel == 'Overall':
        x = estimation_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant']]
    else:
        x = estimation_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant', channel]]
    y = estimation_df['Stage']
    model = sm.Logit(y, x, missing = 'drop')
    result = model.fit()
    logit_models[channel] = result
    print("\nLogit Model for channel - ", channel, "\n", result.summary())

print("\nOLS Model Summaries : \n")

#Estimating OLS Models for each Product Channel
ols_models = {}
for channel in ['Overall', 'Partner Prem', 'Direct Prem', 'Partner Cloud', 'Direct Cloud']:
    if channel == 'Overall':
        x = estimation_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant']]
    else:
        x = estimation_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant', channel]]
    y = estimation_df['Stage']
    model = sm.OLS(y, x, missing = 'drop')
    result = model.fit()
    ols_models[channel] = result
    print("\nOLS Model for channel - ", channel, "\n", result.summary())


Logit Model Summaries : 

Optimization terminated successfully.
         Current function value: 0.436791
         Iterations 7

Logit Model for channel -  Overall 
                            Logit Regression Results                           
Dep. Variable:                  Stage   No. Observations:                  776
Model:                          Logit   Df Residuals:                      768
Method:                           MLE   Df Model:                            7
Date:                Wed, 07 Jun 2023   Pseudo R-squ.:                  0.3657
Time:                        11:10:41   Log-Likelihood:                -338.95
converged:                       True   LL-Null:                       -534.35
Covariance Type:            nonrobust   LLR p-value:                 2.242e-80
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Amount          -1.471e-06   1.32

We choose a 80:20 estimation - holdout split of the dataset, with random sampling.

### Optimizer Function

In [18]:
#Defining a function to optimize discounts based on revenue and each model
def optimize_discount(row, model):
    def revenue(discount):
        return -np.exp(model.params[0] + model.params[1] * row['Amount'] + model.params[2] * discount) / \
               (1 + np.exp(model.params[0] + model.params[1] * row['Amount'] + model.params[2] * discount)) * \
               (row['Amount'] - discount)

    result = minimize_scalar(revenue)
    return result.x

### Calculating expected revenue improvements

In [19]:
#Adding a Revenue column to store Net Amount for leads Won
holdout_df['Revenue'] = np.where(holdout_df['Stage'] == 1, holdout_df['Net Amount'] , 0.0)

#Creating an empty array for Expected Revenue Improvements
expected_revenue_improvements = []

#Evaluating revenue over unoptimized and optimized discounts for Logit Models
for channel, model in logit_models.items():
    if channel == 'Overall':
      x_holdout = holdout_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant']]
    else :
      x_holdout = holdout_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant', channel]]
  
    y_holdout = holdout_df['Stage']
    
    #Calculating the expected revenue over unoptimized discounts
    unoptimized_revenue = np.sum(holdout_df['Net Amount'] * model.predict(x_holdout))
    
    #Calculating the expected revenue over optimized discounts
    holdout_df['Optimized_Discount'] = holdout_df.apply(lambda row: optimize_discount(row, model), axis = 1)
    optimized_revenue = np.sum((holdout_df['Amount'] - holdout_df['Optimized_Discount']) * model.predict(x_holdout))

    #Calculating expected revenue improvement as a ratio    
    unoptimized_eri_ratio = unoptimized_revenue / np.sum(holdout_df['Revenue'])
    optimized_eri_ratio = optimized_revenue / np.sum(holdout_df['Revenue'])
    
    #Appending results to the list
    expected_revenue_improvements.append(('Logit Model', channel, unoptimized_revenue, unoptimized_eri_ratio, optimized_revenue, optimized_eri_ratio))

#Evaluating revenue over unoptimized and optimized discounts for OLS Models
for channel, model in ols_models.items():
    if channel == 'Overall':
      x_holdout = holdout_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant']]
    else :
      x_holdout = holdout_df[['Amount', 'Discount Amount', 'Billing_Plan', 'New Logo', 'Depth', 'End of Quarter', 'Active Days', 'constant', channel]]

    y_holdout = holdout_df['Stage']
    
    #Calculating the expected revenue over unoptimized discounts
    unoptimized_revenue = np.sum(holdout_df['Net Amount'] * model.predict(x_holdout))
    
    #Calculating the expected revenue over optimized discounts
    holdout_df['Optimized_Discount'] = holdout_df.apply(lambda row: optimize_discount(row, model), axis = 1)
    optimized_revenue = np.sum((holdout_df['Amount'] - holdout_df['Optimized_Discount']) * model.predict(x_holdout))

    #Calculating expected revenue improvement as a ratio
    unoptimized_eri_ratio = unoptimized_revenue / np.sum(holdout_df['Revenue'])
    optimized_eri_ratio = optimized_revenue / np.sum(holdout_df['Revenue'])
    
    expected_revenue_improvements.append(('OLS Model', channel, unoptimized_revenue, unoptimized_eri_ratio, optimized_revenue, optimized_eri_ratio))

#Creating the Expected Revenue Improvements Table
expected_revenue_table = pd.DataFrame(expected_revenue_improvements, columns = ['Model', 'Channel', 'Expected Revenue - Unoptimized Discounts', 'ERI Ratio - Unoptimized Discounts', 'Expected Revenue - Optimized Discounts', 'ERI Ratio - Optimized Discounts'])

#Converting expected revenue values to $ Mn format
expected_revenue_table['Expected Revenue - Unoptimized Discounts'] = (expected_revenue_table['Expected Revenue - Unoptimized Discounts'] / 1000000).apply(lambda x: f"${x:.2f} M")
expected_revenue_table['Expected Revenue - Optimized Discounts'] = (expected_revenue_table['Expected Revenue - Optimized Discounts'] / 1000000).apply(lambda x: f"${x:.2f} M")

#Rounding the ERI Ration fields to 3 decimal digits
expected_revenue_table['ERI Ratio - Unoptimized Discounts'] = expected_revenue_table['ERI Ratio - Unoptimized Discounts'].round(3)
expected_revenue_table['ERI Ratio - Optimized Discounts'] = expected_revenue_table['ERI Ratio - Optimized Discounts'].round(3)

#Displaying the Expected Revenue Improvements Table
print("\nExpected Revenue Improvements:")
print(expected_revenue_table)


Expected Revenue Improvements:
         Model        Channel Expected Revenue - Unoptimized Discounts  \
0  Logit Model        Overall                                  $9.35 M   
1  Logit Model   Partner Prem                                  $9.48 M   
2  Logit Model    Direct Prem                                  $9.33 M   
3  Logit Model  Partner Cloud                                  $9.79 M   
4  Logit Model   Direct Cloud                                  $9.06 M   
5    OLS Model        Overall                                  $9.31 M   
6    OLS Model   Partner Prem                                  $9.40 M   
7    OLS Model    Direct Prem                                  $9.31 M   
8    OLS Model  Partner Cloud                                  $9.54 M   
9    OLS Model   Direct Cloud                                  $8.99 M   

   ERI Ratio - Unoptimized Discounts Expected Revenue - Optimized Discounts  \
0                              1.037                               $14.81 

We observe that when discounts are optimized, we see higher expected revenue improvements, as compared to when the discounts are not optimized. The Expected Revenue Improvement Ratio is close to negligible ~1 when the discounts are not optimized while, with the discounts optimized, the ratio is high ~1.6 .

Specifically, use Logit Models while excluding or including the product/channel level, for significant improvement in expected revenue. We see an average revenue improvement ratio of 1.6 for all models, for optimized discounts and 1 for unoptimized discounts.

Based on the expected revenue improvements from the optimized discounts, we can make the following recommendations for the new discounts strategy:

Overall:
The expected revenue with optimized discounts significantly outperforms the unoptimized discounts strategy.

Partner Prem, Direct Prem, Partner Cloud, and Direct Cloud:
For all these product channels, the expected revenue with optimized discounts is higher compared to unoptimized discounts.

Based on these findings, it is recommended to implement the new discounts strategy that optimizes discounts using the respective models for each product channel. This strategy can lead to substantial improvements in revenue generation.

The new approach ensures that discounts are tailored to each specific product channel and customer, taking into account various factors such as the amount, billing plan, new logo, depth, and end of quarter. By optimizing discounts based on these factors, the new strategy aims to strike a balance between maximizing revenue and providing competitive pricing.

As compared to the old approach of using only the 'End-of-Quarter' to adjust discounts can be replaced with the use of above models that consider multiple factors.